**37. ¿Cuál es el contenido al que más se hacen redirecciones? (1)**

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt update
!apt install openjdk-8-jdk-headless -qq
#!apt install default-jre
#!apt install default-jdk
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:14 http://ppa.launchpad.net/graph

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Bajo el csv **redirect_list**


In [ ]:
downloaded = drive.CreateFile({'id':"1XCOf3esuerwAc__OscVww8mtfOZ1LqvM"})
downloaded.GetContentFile('redirect_list.csv') 

In [ ]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

Leo el CSV:

In [ ]:
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('redirect_list.csv', header=True, inferSchema=True)
rdd = df.rdd

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


Le doy un vistazo:

In [ ]:
rdd.count()

1960402

In [ ]:
rdd.take(5)

[Row(rd_from=54, rd_namespace=0, rd_title='Apiales', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=57, rd_namespace=0, rd_title='Atmósfera', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=58, rd_namespace=0, rd_title='Archaea', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=82, rd_namespace=0, rd_title='Agujero_de_gusano', rd_interwiki=None, rd_fragment=None),
 Row(rd_from=108, rd_namespace=0, rd_title='Stipa', rd_interwiki=None, rd_fragment=None)]

rd_from indica el ID del contenido redirigo y rd_tittle el nombre.

Voy a buscar al más redireccionado por el nombre.

In [ ]:
rdd.map(lambda x: (x.rd_title, 1)).reduceByKey(lambda x,y: x+y)\
        .reduce(lambda x,y: x if x[1] > y[1] else y)

('Artículo_futuro', 1161)

Y el top 10 de los mas redireccionados:

In [ ]:
rdd.map(lambda x: (x.rd_title, 1)).reduceByKey(lambda x,y: x+y)\
      .takeOrdered(10, lambda x: -x[1])

[('Artículo_futuro', 1161),
 ('IV_milenio', 1001),
 ('V_milenio', 998),
 ('Estaciones_de_Metrobús_de_la_Ciudad_de_México', 396),
 ('163.117.0.0', 310),
 ('Crataegus_monogyna', 305),
 ('Argelia', 274),
 ('Usuario_retirado', 273),
 ('Historia_de_las_misiones_de_la_provincia_jesuítica_del_Paraguay', 270),
 ('Capsella_bursa-pastoris', 264)]